In [1]:
import skimage

In [2]:
from keras.applications import vgg16
from keras.preprocessing import image
from quiver_engine.server import launch
from keras.models import Model
from keras import datasets
from keras.utils import np_utils
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Input
from keras import backend as K
from keras import optimizers
import os, glob
import numpy as np
import cv2
from matplotlib import pyplot as plt

Using TensorFlow backend.


# MNIST data load and preprocess
- using 2,5,6,9
## training data

In [3]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

In [4]:
# save each number index
idx_2 = np.where(y_train==2)[0]
idx_5 = np.where(y_train==5)[0]
idx_6 = np.where(y_train==6)[0]
idx_9 = np.where(y_train==9)[0]

In [5]:
# make new data
img_2 = X_train[idx_2]
img_5 = X_train[idx_5]
img_6 = X_train[idx_6]
img_9 = X_train[idx_9]

lab_2 = np.zeros((len(img_2), 4))
lab_2[:,0] = 1.
                 
lab_5 = np.zeros((len(img_5), 4))
lab_5[:,1] = 1.

lab_6 = np.zeros((len(img_6), 4))
lab_6[:,2] = 1.

lab_9 = np.zeros((len(img_9), 4))
lab_9[:,3] = 1.

In [6]:
# resize
re_img2= np.zeros((len(idx_2),120,120))
for i in range(0,len(idx_2)):
    
    img =img_2[i] 
    re_img2[i] = cv2.resize(img,(120,120), interpolation=cv2.INTER_AREA)

re_img5= np.zeros((len(idx_5),120,120))
for i in range(0,len(idx_5)):
    img =img_5[i] 
    re_img5[i] = cv2.resize(img,(120,120), interpolation=cv2.INTER_AREA)             
re_img6= np.zeros((len(idx_6),120,120))
for i in range(0,len(idx_6)):
    img =img_6[i] 
    re_img6[i] = cv2.resize(img,(120,120), interpolation=cv2.INTER_AREA)
    
re_img9= np.zeros((len(idx_9),120,120))
for i in range(0,len(idx_9)):
    img =img_9[i] 
    re_img9[i] = cv2.resize(img,(120,120), interpolation=cv2.INTER_AREA)

In [7]:
# make input data
dnum = 30
data_list = ['img2','img5','img6','img9' ]
lab_list = ['lab2', 'lab5','lab6','lab9']

data_dic = {'img2':re_img2[0:dnum], 'img5':re_img5[0:dnum],'img6':re_img6[0:dnum],'img9':re_img9[0:dnum]}
lab_dic = {'lab2':lab_2[0:dnum],'lab5':lab_5[0:dnum],'lab6':lab_6[0:dnum],'lab9':lab_9[0:dnum]}

new_x_train = np.zeros((dnum*4,120,120))
new_y_train = np.zeros((dnum*4,4))

for i in range(4):
    img = data_list[i]
    lab= lab_list[i]
    new_x_train[i*dnum:(i+1)*dnum] = data_dic[img]
    new_y_train[i*dnum:(i+1)*dnum] = lab_dic[lab]

In [8]:
color_x_train = np.zeros((len(new_x_train), 120,120,3))
#expand dimension & GRAY to RGB
for i in range(dnum*4):
    tmp = np.expand_dims(new_x_train[i],axis=3)
    color_x_train[i] = skimage.color.gray2rgb(new_x_train[i]).astype(np.uint8)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  after removing the cwd from sys.path.


In [ ]:
plt.imshow(color_x_train[50])

In [ ]:
#check 
i = 11900
plt.imshow(color_x_train[i])
plt.title('{}'.format(new_y_train[i]))
plt.show()


## Test data

In [9]:
dnum_t = 100
# save each number index
idx_2t = np.where(y_test==2)[0]
idx_5t = np.where(y_test==5)[0]
idx_6t = np.where(y_test==6)[0]
idx_9t = np.where(y_test==9)[0]

# make new data
img_2t = X_test[idx_2t]
img_5t = X_test[idx_5t]
img_6t = X_test[idx_6t]
img_9t = X_test[idx_9t]

lab_2t = np.zeros((len(img_2t), 4))
lab_2t[:,0] = 1.
                 
lab_5t = np.zeros((len(img_5t), 4))
lab_5t[:,1] = 1.

lab_6t = np.zeros((len(img_6t), 4))
lab_6t[:,2] = 1.

lab_9t = np.zeros((len(img_9t), 4))
lab_9t[:,3] = 1.


# resize
re_img2t= np.zeros((len(idx_2t),120,120))
for i in range(0,len(idx_2t)):
    
    imgt =img_2t[i] 
    re_img2t[i] = cv2.resize(imgt,(120,120), interpolation=cv2.INTER_AREA)

re_img5t= np.zeros((len(idx_5t),120,120))
for i in range(0,len(idx_5t)):
    imgt =img_5t[i] 
    re_img5t[i] = cv2.resize(imgt,(120,120), interpolation=cv2.INTER_AREA)             
re_img6t= np.zeros((len(idx_6t),120,120))
for i in range(0,len(idx_6t)):
    imgt =img_6t[i] 
    re_img6t[i] = cv2.resize(imgt,(120,120), interpolation=cv2.INTER_AREA)
    
re_img9t= np.zeros((len(idx_9t),120,120))
for i in range(0,len(idx_9t)):
    imgt =img_9t[i] 
    re_img9t[i] = cv2.resize(imgt,(120,120), interpolation=cv2.INTER_AREA)
    
# make input data
data_list_t = ['img2t','img5t','img6t','img9t' ]
lab_list_t = ['lab2t', 'lab5t','lab6t','lab9t']

data_dic_t = {'img2t':re_img2t[0:dnum_t], 'img5t':re_img5t[0:dnum_t],'img6t':re_img6t[0:dnum_t],'img9t':re_img9t[0:dnum_t]}
lab_dic_t = {'lab2t':lab_2t[0:dnum_t],'lab5t':lab_5t[0:dnum_t],'lab6t':lab_6t[0:dnum_t],'lab9t':lab_9t[0:dnum_t]}

new_x_test = np.zeros((dnum_t*4,120,120))
new_y_test = np.zeros((dnum_t*4,4))

for i in range(4):
    imgt = data_list_t[i]
    labt= lab_list_t[i]
    new_x_test[i*dnum_t:(i+1)*dnum_t] = data_dic_t[imgt]
    new_y_test[i*dnum_t:(i+1)*dnum_t] = lab_dic_t[labt]
    
color_x_test = np.zeros((len(new_x_test), 120,120,3))
#expand dimension & GRAY to RGB
for i in range(dnum_t*4):
    tmp = np.expand_dims(new_x_test[i],axis=3)
    color_x_test[i] = skimage.color.gray2rgb(new_x_test[i]).astype(np.uint8)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:67: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


In [ ]:
plt.imshow(re_img6t[0])

In [ ]:
plt.figure()
plt.imshow(color_x_test[399].astype(np.uint8))
plt.figure()
plt.imshow(color_x_test[200])

## Model
- 기존 VGG16 모델에서 fully connected부분을 제외하고 가져옴
- Flatten함수-> 추가하여 fully connected할 때 곱할 개수 구함
- Dense 함수 -> shape이 (None, 512)인 layer 생성, classification layer(4) 생성

In [10]:
# create the base pre-trained model
base_model = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(120,120,3))
x = base_model.output
x = Flatten(name='flatten')(x) # fully connected할 때 곱할 값을 구함

#x = Dense(2048, activation='relu', name='dense_1')(x)
x = Dense(512, activation='relu', name='dense_2')(x)
#predictions = Dense(5, activation='softmax', name='prediction')(x)
predictions = Dense(4, activation='softmax', name='prediction')(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

In [11]:
#Randomly shuffle data and labels from different files in the same order
idx = np.random.permutation(len(color_x_train))
X_s,Y_s = color_x_train[idx], new_y_train[idx]
#X_t,Y_t = color_x_test[idx], new_y_test[idx]

- loss : categorical_crossentropy
- optimizer : Adam(Learning Rate : 0.0001)

In [12]:
#train
model.compile(loss = 'categorical_crossentropy', optimizer = optimizers.Adam(lr=0.0001),metrics=["accuracy"])

In [13]:
history = model.fit(X_s,Y_s,epochs=10,batch_size=30, validation_split=0.1)

Train on 108 samples, validate on 12 samples
Epoch 1/10
108/108 [==============================] - 2s 16ms/step - loss: 5.4817 - acc: 0.3426 - val_loss: 2.1140 - val_acc: 0.5833
Epoch 2/10
108/108 [==============================] - 0s 1ms/step - loss: 1.9662 - acc: 0.6944 - val_loss: 0.3209 - val_acc: 0.9167
Epoch 3/10
108/108 [==============================] - 0s 1ms/step - loss: 0.6960 - acc: 0.8981 - val_loss: 0.0036 - val_acc: 1.0000
Epoch 4/10
108/108 [==============================] - 0s 1ms/step - loss: 0.4215 - acc: 0.9444 - val_loss: 0.0063 - val_acc: 1.0000
Epoch 5/10
108/108 [==============================] - 0s 1ms/step - loss: 0.2182 - acc: 0.9630 - val_loss: 0.0290 - val_acc: 1.0000
Epoch 6/10
108/108 [==============================] - 0s 1ms/step - loss: 0.0379 - acc: 0.9815 - val_loss: 0.0318 - val_acc: 1.0000
Epoch 7/10
108/108 [==============================] - 0s 1ms/step - loss: 4.3517e-04 - acc: 1.0000 - val_loss: 0.0363 - val_acc: 1.0000
Epoch 8/10
108/108 [======

In [14]:
score = model.evaluate(color_x_test, new_y_test,batch_size=30)

400/400 [==============================] - 1s 1ms/step


In [15]:
print(score[0])
print(score[1])

1.12638277358
0.804999998212


In [ ]:
plt.plot(history.history['loss'], 'r-')
plt.plot(history.history['val_loss'], 'b-.')
plt.legend(['loss', 'val_loss'], loc=1)
plt.show()